In [15]:
### sample dataset

from datasets import Dataset
data = {
    "prompt": [
        "What is the capital of France?",
        "Define gravity.",
        "Who wrote Hamlet?"
    ],
    "response": [
        "The capital of France is Paris.",
        "Gravity is a force that attracts two bodies toward each other.",
        "William Shakespeare wrote Hamlet."
    ]
}

dataset = Dataset.from_dict(data)
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']


def format(example):
    return {"text": f"### Instruction:\n{example['prompt']}\n\n### Response:\n{example['response']}"}

train_dataset = train_dataset.map(format)
eval_dataset = eval_dataset.map(format)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [16]:
### load model and tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "sshleifer/tiny-gpt2" #Tiny demo model

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model= AutoModelForCausalLM.from_pretrained(model_name)

In [17]:
### Apply QLORA and PEFT
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r = 4,
    lora_alpha = 8,
    target_modules = ["c_attn"],  ### for gp2 architecture
    lora_dropout = 0.05,
    task_type= TaskType.CAUSAL_LM    
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 64 || all params: 102,778 || trainable%: 0.0623


In [28]:
#### Tokenizer and training configration

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length" ,max_length=64)

train_dataset= train_dataset.map(tokenize)
eval_dataset = eval_dataset.map(tokenize)


training_arguments = TrainingArguments(
    output_dir = "qlora_out",
    per_device_train_batch_size = 2,
    num_train_epochs = 3,
    logging_steps=1,
    save_steps= 2,
    save_total_limit = 2,
    fp16=False ##CPU only
    )

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer , mlm=False)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    data_collator= data_collator
)
trainer.train()

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/vishal/envs/rnd/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,10.400300
2,10.400200
3,10.401100


/home/vishal/envs/rnd/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3, training_loss=10.400525410970053, metrics={'train_runtime': 3.0599, 'train_samples_per_second': 1.961, 'train_steps_per_second': 0.98, 'total_flos': 497664.0, 'train_loss': 10.400525410970053, 'epoch': 3.0})

In [29]:
model.save_pretrained("qlora_out/qlora-tiny-model")
tokenizer.save_pretrained("qlora_out/qlora-tiny-model")

('qlora_out/qlora-tiny-model/tokenizer_config.json',
 'qlora_out/qlora-tiny-model/special_tokens_map.json',
 'qlora_out/qlora-tiny-model/vocab.json',
 'qlora_out/qlora-tiny-model/merges.txt',
 'qlora_out/qlora-tiny-model/added_tokens.json',
 'qlora_out/qlora-tiny-model/tokenizer.json')

In [35]:
### Use pipline for inferanceing
from transformers import pipeline

pipe = pipeline("text-generation", model ="qlora_out/qlora-tiny-model", tokenizer = tokenizer)

Device set to use cpu


In [36]:
result = pipe("### Instruction:\nWho discovered penicillin?\n\n### Response:\n", max_new_tokens=50)
print(result[0]["generated_text"])

### Instruction:
Who discovered penicillin?

### Response:
hibit reviewing heirJD Rh subst Daniel vendors stairs Motorola stairs Motorolaatisf subst Rh Brew reborn TAhibitpress confir dispatch Hancock stairs credibility004 Habitoother Amph Money TA autonomy dispatch dispatchikendit heir antibioticpress confir Hancock autonomy Hancock reborn subst Amph confir scalp ProbRocket
